<center>
<img src="../../img/ods_stickers.jpg">
## Открытый курс по машинному обучению. Сессия №3
<center>Автор материала: программист-исследователь Mail.Ru Group Юрий Кашницкий

# <center> Домашнее задание № 8
## <center> Vowpal Wabbit в задаче классификации тегов вопросов на Stackoverflow

## План
    1. Введение
    2. Описание данных
    3. Предобработка данных
    4. Обучение и проверка моделей
    5. Заключение

### 1. Введение

В этом задании вы будете делать примерно то же, что я каждую неделю –  в Mail.Ru Group: обучать модели на выборке в несколько гигабайт. Задание можно выполнить и на Windows с Python, но я рекомендую поработать под \*NIX-системой (например, через Docker) и активно использовать язык bash.
Немного снобизма (простите, но правда): если вы захотите работать в лучших компаниях мира в области ML, вам все равно понадобится опыт работы с bash под UNIX.

[Веб-форма](https://docs.google.com/forms/d/1VaxYXnmbpeP185qPk2_V_BzbeduVUVyTdLPQwSCxDGA/edit) для ответов.

Для выполнения задания понадобится установленный Vowpal Wabbit (уже есть в докер-контейнере курса, см. инструкцию в Wiki [репозитория](https://github.com/Yorko/mlcourse_open) нашего курса) и примерно 70 Гб дискового пространства. Я тестировал решение не на каком-то суперкомпе, а на Macbook Pro 2015 (8 ядер, 16 Гб памяти), и самая тяжеловесная модель обучалась около 12 минут, так что задание реально выполнить и с простым железом. Но если вы планируете когда-либо арендовать сервера Amazon, можно попробовать это сделать уже сейчас.

Материалы в помощь:
 - интерактивный [тьюториал](https://www.codecademy.com/en/courses/learn-the-command-line/lessons/environment/exercises/bash-profile) CodeAcademy по утилитам командной строки UNIX (примерно на час-полтора)
 - [статья](https://habrahabr.ru/post/280562/) про то, как арендовать на Amazon машину (еще раз: это не обязательно для выполнения задания, но будет хорошим опытом, если вы это делаете впервые)

### 2. Описание данных

Имеются 10 Гб вопросов со StackOverflow – [скачайте](https://drive.google.com/file/d/1ZU4J3KhJDrHVMj48fROFcTsTZKorPGlG/view) и распакуйте архив. 

Формат данных простой:<br>
<center>*текст вопроса* (слова через пробел) TAB *теги вопроса* (через пробел)

Здесь TAB – это символ табуляции.
Пример первой записи в выборке:

In [1]:
!head -1 stackoverflow.10kk.tsv

 is there a way to apply a background color through css at the tr level i can apply it at the td level like this my td background color e8e8e8 background e8e8e8 however the background color doesn t seem to get applied when i attempt to apply the background color at the tr level like this my tr background color e8e8e8 background e8e8e8 is there a css trick to making this work or does css not natively support this for some reason 	css css3 css-selectors


Здесь у нас текст вопроса, затем табуляция и теги вопроса: *css, css3* и *css-selectors*. Всего в выборке таких вопросов 10 миллионов. 

In [2]:
%%time
!wc -l stackoverflow.10kk.tsv

 10000000 stackoverflow.10kk.tsv
CPU times: user 186 ms, sys: 71.9 ms, total: 258 ms
Wall time: 12.4 s


Обратите внимание на то, что такие данные я уже не хочу загружать в оперативную память и, пока можно, буду пользоваться эффективными утилитами UNIX –  head, tail, wc, cat, cut и прочими.

### 3. Предобработка данных

Давайте выберем в наших данных все вопросы с тегами *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift* и подготовим обучающую выборку в формате Vowpal Wabbit. Будем решать задачу 10-классовой классификации вопросов по перечисленным тегам.

Вообще, как мы видим, у каждого вопроса может быть несколько тегов, но мы упростим себе задачу и будем у каждого вопроса выбирать один из перечисленных тегов либо игнорировать вопрос, если таковых тегов нет. 
Но вообще VW поддерживает multilabel classification (аргумент  --multilabel_oaa).
<br>
<br>
Реализуйте в виде отдельного файла `preprocess.py` код для подготовки данных. Он должен отобрать строки, в которых есть перечисленные теги, и переписать их в отдельный файл в формат Vowpal Wabbit. Детали:
 - скрипт должен работать с аргументами командной строки: с путями к файлам на входе и на выходе
 - строки обрабатываются по одной (можно использовать tqdm для подсчета числа итераций)
 - если табуляций в строке нет или их больше одной, считаем строку поврежденной и пропускаем
 - в противном случае смотрим, сколько в строке тегов из списка *javascript, java, python, ruby, php, c++, c#, go, scala* и  *swift*. Если ровно один, то записываем строку в выходной файл в формате VW: `label | text`, где `label` – число от 1 до 10 (1 - *javascript*, ... 10 – *swift*). Пропускаем те строки, где интересующих тегов больше или меньше одного 
 - из текста вопроса надо выкинуть двоеточия и вертикальные палки, если они есть – в VW это спецсимволы

In [49]:
from tqdm import tqdm

tags = ['javascript', 'java', 'python', 'ruby', 'php', 'c++', 'c#', 'go', 'scala', 'swift']

with open('stackoverflow.10kk.tsv') as f, open('stackoverflow.vw', 'w') as to_f:
    for line in tqdm(f):
        splited_line = line.split('\t')
        if len(splited_line) != 2:
            continue

        text, text_tags = splited_line
        text = text.replace('|', ' ').replace(':', ' ').replace('  ', ' ')

        text_tags = set(text_tags.replace('\n', '').split())
        need_tags = set(tags).intersection(text_tags)
        if len(need_tags) != 1:
            continue

        tag_index = tags.index(need_tags.pop()) + 1

        to_f.write('%d |text %s\n' % (tag_index, text))

10000000it [01:49, 91507.34it/s]


In [7]:
import os
from tqdm import tqdm
from time import time
import numpy as np
from sklearn.metrics import accuracy_score

Должно получиться вот такое число строк – 4389054. 10 Гб у меня обработались примерно за 2 минуты.

In [48]:
# !python3.5 preprocess.py stackoverflow.10kk.tsv stackoverflow.vw

In [51]:
%%time
!wc -l stackoverflow.vw

 4389058 stackoverflow.vw
CPU times: user 82.1 ms, sys: 35 ms, total: 117 ms
Wall time: 5.42 s


In [6]:
# !gzip stackoverflow.10kk.tsv

Поделите выборку на обучающую, проверочную и тестовую части в равной пропорции - по  1463018 в каждый файл. Перемешивать не надо, первые 1463018 строк должны пойти в обучающую часть `stackoverflow_train.vw`, последние 1463018 – в тестовую `stackoverflow_test.vw`, оставшиеся – в проверочную `stackoverflow_valid.vw`. 

Также сохраните векторы ответов для проверочной и тестовой выборки в отдельные файлы `stackoverflow_valid_labels.txt` и `stackoverflow_test_labels.txt`.

Тут вам помогут утилиты `head`, `tail`, `split`, `cat` и `cut`.

In [50]:
!head -n 1 stackoverflow.vw

1 |text  i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if load_more_staff true loadmorestaff what am i doing wrong can you only fire one event from window scroll i assume no

In [ ]:
# один из примеров разделения.
# заменил на stplit

# !head -n 1463018 stackoverflow.vw > stackoverflow_train.vw
# !tail -n 1463018 stackoverflow.vw > stackoverflow_test.vw
# awk 'NR==1463018'

In [52]:
!split -l 1463018 stackoverflow.vw stackoverflow_

In [53]:
!ls

a
hw8_session3_vw_stackoverflow_tags_10mln.ipynb
prepare.py
stackoverflow.10kk.tsv
stackoverflow.vw
stackoverflow_aa
stackoverflow_ab
stackoverflow_ac
stackoverflow_ad


In [54]:
!mv stackoverflow_aa stackoverflow_train.vw
!mv stackoverflow_ab stackoverflow_valid.vw
!mv stackoverflow_ac stackoverflow_test.vw
!rm stackoverflow_ad

In [55]:
!cat stackoverflow_valid.vw | cut -c 1 > stackoverflow_valid_labels.txt
!cat stackoverflow_test.vw | cut -c 1 > stackoverflow_test_labels.txt

In [57]:
!head -n 2 stackoverflow_train.vw

1 |text  i ve got some code in window scroll that checks if an element is visible then triggers another function however only the first section of code is firing both bits of code work in and of themselves if i swap their order whichever is on top fires correctly my code is as follows fn isonscreen function use strict var win window viewport top win scrolltop left win scrollleft bounds this offset viewport right viewport left + win width viewport bottom viewport top + win height bounds right bounds left + this outerwidth bounds bottom bounds top + this outerheight return viewport right lt bounds left viewport left gt bounds right viewport bottom lt bounds top viewport top gt bounds bottom window scroll function use strict var load_more_results ajax load_more_results isonscreen if load_more_results true loadmoreresults var load_more_staff ajax load_more_staff isonscreen if load_more_staff true loadmorestaff what am i doing wrong can you only fire one event from window scroll i assume no

In [59]:
!head -n 2 stackoverflow_valid_labels.txt

2
7


### 4. Обучение и проверка моделей

Обучите Vowpal Wabbit на выборке `stackoverflow_train.vw` 9 раз, перебирая параметры passes (1,3,5), ngram (1,2,3).
Остальные параметры укажите следующие: `loss_function=hinge`, `bit_precision`=28 и `seed`=17. Также скажите VW, что это 10-классовая задача.

Проверяйте долю правильных ответов на выборке `stackoverflow_valid.vw`. Выберите лучшую модель и проверьте качество на выборке `stackoverflow_test.vw`.

In [27]:
!vw --help

Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = 
num sources = 1

VW options:
  --random_seed arg                     seed random number generator
  --ring_size arg                       size of example ring
  --onethread                           Disable parse thread

Update options:
  -l [ --learning_rate ] arg            Set learning rate
  --power_t arg                         t power value
  --decay_learning_rate arg             Set Decay factor for learning_rate 
                                        between passes
  --initial_t arg                       initial t value
  --feature_mask arg                    Use existing regressor to determine 
                                        which parameters may be updated.  If no
                                        initial_regressor given, also used for 
                                        initial weights.

Weight options:
  -i [ --initial_regressor ] arg        Initial r

In [15]:
!vw --oaa 10 --data stackoverflow_train.vw -f vw.model --loss_function=hinge
# --oaa 10 - 10 классов

final_regressor = vw.model
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = stackoverflow_train.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.000000 0.000000            1            1.0        1        1      161
0.500000 1.000000            2            2.0        4        1       68
0.750000 1.000000            4            4.0        7        1       88
0.750000 0.750000            8            8.0        7        1       95
0.812500 0.875000           16           16.0        7        7      209
0.812500 0.812500           32           32.0        7        5      174
0.765625 0.718750           64           64.0        3        3      204
0.664062 0.562500          128          128.0        1        5       29
0.621094 0.578125          256          256.0        5        3      169
0.558594 0.496094          5

In [33]:
!ls

a
hw8_session3_vw_stackoverflow_tags_10mln.ipynb
prepare.py
stackoverflow.10kk.tsv
stackoverflow.vw
stackoverflow_test.vw
stackoverflow_test_labels.txt
stackoverflow_train.vw
stackoverflow_valid.vw
stackoverflow_valid_labels.txt
vw.model


In [34]:
!tail vw.model

   8.5.0     m                       (    --hash_seed 0 --oaa 10 --link identity    '� 

<font color='red'> Вопрос 1.</font> Какое сочетание параметров дает наибольшую долю правильных ответов на проверочной выборке `stackoverflow_valid.vw`?
- Биграммы и 3 прохода по выборке
- Триграммы и 5 проходов по выборке
- Биграммы и 1 проход по выборке
- Униграммы и 1 проход по выборке

In [29]:
import os
import numpy as np
from sklearn.metrics import accuracy_score

def get_accuracy(valid_labels_file='stackoverflow_valid_labels.txt'):
    vw_pred = np.loadtxt('predicted.txt')
    test_labels = np.loadtxt(valid_labels_file)
    return accuracy_score(test_labels, vw_pred)

In [20]:
# (--ngram 2) биграммы и (--passes 3) 3 прохода
!vw --oaa 10 --ngram 2 --passes 3 --cache_file \
--data stackoverflow_train.vw -f vw.model --loss_function=hinge --quiet

!vw -i vw.model -t -d stackoverflow_valid.vw -p predicted.txt --quiet

get_accuracy()  # 0.84809

In [25]:
# Триграммы и 5 проходов по выборке

!vw --oaa 10 --ngram 3 --passes 5 --cache_file \
--data stackoverflow_train.vw -f vw.model --loss_function=hinge --quiet

!vw -i vw.model -t -d stackoverflow_valid.vw -p predicted.txt --quiet

get_accuracy()

0.8302392725174946

In [26]:
# Биграммы и 1 проход по выборке

!vw --oaa 10 --ngram 2 --passes 1 --cache_file \
--data stackoverflow_train.vw -f vw.model --loss_function=hinge --quiet

!vw -i vw.model -t -d stackoverflow_valid.vw -p predicted.txt --quiet

get_accuracy()

0.8480244262203199

In [38]:
# Униграммы и 1 проход по выборке
!vw --oaa 10 --ngram 1 --passes 1 --cache_file \
--data stackoverflow_train.vw -f vw.model --loss_function=hinge --quiet

!vw -i vw.model -t -d stackoverflow_valid.vw -p predicted.txt --quiet

get_accuracy()

0.8610495564647872

Проверьте лучшую (по доле правильных ответов на валидации) модель на тестовой выборке. 

### Ответ 1: Униграммы и 1 проход по выборке

<font color='red'> Вопрос 2.</font> Как соотносятся доли правильных ответов лучшей (по доле правильных ответов на валидации) модели на проверочной и на тестовой выборках? (здесь % – это процентный пункт, т.е., скажем, снижение с 50% до 40% – это на 10%, а не 20%).
- На тестовой ниже примерно на 2%
- На тестовой ниже примерно на 3%
- Результаты почти одинаковы – отличаются меньше чем на 0.5%

In [30]:
!vw -i vw.model -t -d stackoverflow_valid.vw -p predicted.txt --quiet
get_accuracy()

0.8610495564647872

In [31]:
!vw -i vw.model -t -d stackoverflow_test.vw -p predicted.txt --quiet
get_accuracy('stackoverflow_test_labels.txt')

0.8611356798070837

### Ответ 2: Результаты почти одинаковы – отличаются меньше чем на 0.5%

Обучите VW с параметрами, подобранными на проверочной выборке, теперь на объединении обучающей и проверочной выборок. Посчитайте долю правильных ответов на тестовой выборке. 

In [32]:
!cat stackoverflow_train.vw > stackoverflow_train_and_test.vw
!cat stackoverflow_valid.vw >> stackoverflow_train_and_test.vw

In [36]:
!wc stackoverflow_train_and_test.vw

 2926036 586648450 3386159387 stackoverflow_train_and_test.vw


In [39]:
!rm vw.model
!rm predicted.txt

# best model
!vw --oaa 10 --ngram 1 --passes 1 --cache_file \
--data stackoverflow_train_and_test.vw -f vw.model --loss_function=hinge --quiet

!vw -i vw.model -t -d stackoverflow_test.vw -p predicted.txt --quiet

get_accuracy('stackoverflow_test_labels.txt')

0.8611356798070837

In [35]:
0.8611356798070837 - 0.8611356798070837

0.0

<font color='red'> Вопрос 3.</font> На сколько процентных пунктов повысилась доля правильных ответов модели после обучения на вдвое большей выборке (обучающая `stackoverflow_train.vw` + проверочная `stackoverflow_valid.vw`) по сравнению с моделью, обученной только на `stackoverflow_train.vw`?
 - 0.1%
 - 0.4%
 - 0.8%
 - 1.2%

### Вопрос 3: 0.1%  (хз почему) хотя 0.0

### 5. Заключение

В этом задании мы только познакомились с Vowpal Wabbit. Что еще можно попробовать:
 - Классификация с несколькими ответами (multilabel classification, аргумент  `multilabel_oaa`) – формат данных тут как раз под такую задачу
 - Настройка параметров VW с hyperopt, авторы библиотеки утверждают, что качество должно сильно зависеть от параметров изменения шага градиентного спуска (`initial_t` и `power_t`). Также можно потестировать разные функции потерь – обучать логистическую регресиию или линейный SVM
 - Познакомиться с факторизационными машинами и их реализацией в VW (аргумент `lrq`)